In [10]:
import great_expectations as gx
from great_expectations.checkpoint import Checkpoint

# Create a DataContext as an entry point to the GX Python API
context = gx.get_context()

In [11]:
datasource_name = "giang-nyctaxi"
my_connection_string = (
    "postgresql+psycopg2://k6:k6@localhost:5432/k6"
)

pg_datasource = context.sources.add_postgres(
    name=datasource_name, connection_string=my_connection_string
)

DataContextError: Can not write the fluent datasource giang-nyctaxi because a datasource of that name already exists in the data context.

In [12]:
pg_datasource.add_table_asset(
    name="postgres_taxi_data", table_name="nyc_taxi_test", schema_name="staging"
)

TestConnectionError: Attempt to connect to table: staging.nyc_taxi_test failed because the table nyc_taxi_test does not exist.

In [13]:
batch_request = pg_datasource.get_asset("postgres_taxi_data").build_batch_request()

In [14]:
expectation_suite_name = "validate_trip_data"
context.add_or_update_expectation_suite(expectation_suite_name=expectation_suite_name)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

print(validator.head())

KeyboardInterrupt: 

In [15]:
# Add the same expectations as the single-file
validator.expect_column_values_to_not_be_null("vendor_id")
validator.expect_column_values_to_not_be_null("rate_code_id")
validator.expect_column_values_to_not_be_null("dropoff_location_id")
validator.expect_column_values_to_not_be_null("pickup_location_id")
validator.expect_column_values_to_not_be_null("payment_type_id")
validator.expect_column_values_to_not_be_null("service_type")
validator.expect_column_values_to_not_be_null("pickup_latitude")
validator.expect_column_values_to_not_be_null("pickup_longitude")
validator.expect_column_values_to_not_be_null("dropoff_latitude")
validator.expect_column_values_to_not_be_null("dropoff_longitude")

validator.expect_column_values_to_be_between("trip_distance", min_value=0, max_value=100)
validator.expect_column_values_to_be_between("extra", min_value=0, max_value=3)

validator.save_expectation_suite(discard_failed_expectations=False)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
# Similar to a single file, create a checkpoint to validate the result
# Define the checkpoint
checkpoint = context.add_or_update_checkpoint(
    name="staging_tripdata_asset_checkpoint",
    validator=validator
)

# Get the result after validator
checkpoint_result = checkpoint.run()

# Quick view on the validation result
context.view_validation_result(checkpoint_result)